# Instructions

1. Go to https://colab.research.google.com and choose the \"Upload\" option to upload this notebook file.
1. In the Edit menu, choose \"Notebook Settings\" and then set the \"Hardware Accelerator\" dropdown to GPU.
1. Read through the code in the following sections:
  * [IMDB Dataset](#scrollTo=mXcb24B6a03_)
  * [Define model](#scrollTo=kAz68ipVa05_)
  * [Train model](#scrollTo=kIynp1v_a06Y)
  * [Assess model](#scrollTo=ALyNCqx4a06r)
1. Complete at least one of these exercises. Remember to keep notes about what you do!
  * [Exercise Option #1 - Standard Difficulty](#scrollTo=_9dsjJwya06_)
  * [Exercise Option #2 - Advanced Difficulty](#scrollTo=nyZbljLAa09z)

## Documentation/Sources
* [Class Notes](https://jennselby.github.io/MachineLearningCourseNotes/#recurrent-neural-networks)
* [https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/) for information on sequence classification with keras
* [https://keras.io/](https://keras.io/) Keras API documentation
* [Keras recurrent tutorial](https://github.com/Vict0rSch/deep_learning/tree/master/keras/recurrent)

In [1]:
# # upgrade tensorflow to tensorflow 2
# %tensorflow_version 2.x
# display matplotlib plots
%matplotlib inline
from tensorflow import test
from tensorflow import device

# IMDB Dataset
The [IMDB dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) consists of movie reviews (x_train) that have been marked as positive or negative (y_train). See the [Word Vectors Tutorial](https://github.com/jennselby/MachineLearningTutorials/blob/master/WordVectors.ipynb) for more details on the IMDB dataset.

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [3]:
(imdb_x_train, imdb_y_train), (imdb_x_test, imdb_y_test) = imdb.load_data()

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\brandon\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\brandon\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprec

For a standard keras model, every input has to be the same length, so we need to set some length after which we will cutoff the rest of the review. (We will also need to pad the shorter reviews with zeros to make them the same length).

In [4]:
cutoff = 500
imdb_x_train_padded = sequence.pad_sequences(imdb_x_train, maxlen=cutoff)
imdb_x_test_padded = sequence.pad_sequences(imdb_x_test, maxlen=cutoff)

 # see https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset
imdb_index_offset = 3

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define model

Unlike last time, when we used convolutional layers, we're going to use an LSTM, a special type of recurrent network.

Using recurrent networks means that rather than seeing these reviews as one input happening all at once, with the convolutional layers taking into account which words are next to each other, we are going to see them as a sequence of inputs, with one word occurring at each timestep.

In [42]:
imdb_lstm_model = Sequential()
imdb_lstm_model.add(Embedding(input_dim=len(imdb.get_word_index()) + imdb_index_offset,
                              output_dim=100,
                              input_length=cutoff))
# return_sequences tells the LSTM to output the full sequence, for use by the next LSTM layer. The final
# LSTM layer should return only the output sequence, for use in the Dense output layer
imdb_lstm_model.add(LSTM(units=32, return_sequences=True))
imdb_lstm_model.add(LSTM(units=32))
imdb_lstm_model.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
imdb_lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

# Train model

In [43]:
# Train using GPU acceleration
# (see https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=Y04m-jvKRDsJ)
device_name = test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

with device('/device:GPU:0'):
  imdb_lstm_model.fit(imdb_x_train_padded, imdb_y_train, epochs=1, batch_size=64)

391/391 [==============================] - 153s 384ms/step - loss: 0.4777 - binary_accuracy: 0.7468


# Assess model

In [44]:
with device('/device:GPU:0'):
  imdb_lstm_scores = imdb_lstm_model.evaluate(imdb_x_test_padded, imdb_y_test)
  print('loss: {} accuracy: {}'.format(*imdb_lstm_scores))

782/782 [==============================] - 70s 88ms/step - loss: 0.2957 - binary_accuracy: 0.8780
loss: 0.2957181930541992 accuracy: 0.878000020980835


# Exercise Option #1 - Standard Difficulty

Experiment with different model configurations from the one above. Try other recurrent layers, different numbers of layers, change some of the defaults. See [Keras Recurrent Layers](https://keras.io/layers/recurrent/)

__Keep notes on what you try and what results you get.__

## Tested changes
- Add a dropout layer after the second LSTM layer with drop rate 0.5 to prevent overfitting
    - Decreases accuracy as overfitting doesn't seem to be a problem with the current network design.
- Add a **time-distributed** dense layer of 100 neurons in the feed-forward section of the network
    - Ran out of memory when allocating a \[32000, 88587\] matrix in GPU memory.
- Changing the dimensionality of the word vector embeddings from 100.
    - Decreasing dimensionality seems to have a negligible impact on accuracy—at what point does the word vector become too low-dimensional to encode information accurately?

In [39]:
from tensorflow.keras.layers import Dropout, TimeDistributed

model_1 = Sequential()
model_1.add(Embedding(input_dim=len(imdb.get_word_index()) + imdb_index_offset,
                              output_dim=32, # Decreasing word vector dimensionality.
                              input_length=cutoff))
# return_sequences tells the LSTM to output the full sequence, for use by the next LSTM layer. The final
# LSTM layer should return only the output sequence, for use in the Dense output layer
model_1.add(LSTM(units=32, return_sequences=True))
model_1.add(LSTM(units=32))
model_1.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

with device('/device:GPU:0'):
  model_1.fit(imdb_x_train_padded, imdb_y_train, epochs=1, batch_size=64)

391/391 [==============================] - 139s 347ms/step - loss: 0.5334 - binary_accuracy: 0.7016


In [40]:
with device('/device:GPU:0'):
  model_1_scores = model_1.evaluate(imdb_x_test_padded, imdb_y_test)
  print('loss: {} accuracy: {}'.format(*model_1_scores))

782/782 [==============================] - 60s 76ms/step - loss: 0.3312 - binary_accuracy: 0.8530
loss: 0.3312091827392578 accuracy: 0.8529999852180481


## Model 2

- Decreasing the word vector dimensionality to 8 produces a similar accuracy rate of ~87% to the results observed with a 100-dimensional word vector.

In [45]:
model_2 = Sequential()
model_2.add(Embedding(input_dim=len(imdb.get_word_index()) + imdb_index_offset,
                              output_dim=8,
                              input_length=cutoff))
# return_sequences tells the LSTM to output the full sequence, for use by the next LSTM layer. The final
# LSTM layer should return only the output sequence, for use in the Dense output layer
model_2.add(LSTM(units=32, return_sequences=True))
model_2.add(LSTM(units=32))
model_2.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

with device('/device:GPU:0'):
  model_2.fit(imdb_x_train_padded, imdb_y_train, epochs=1, batch_size=64)

391/391 [==============================] - 142s 354ms/step - loss: 0.5317 - binary_accuracy: 0.7008


In [46]:
with device('/device:GPU:0'):
  model_2_scores = model_2.evaluate(imdb_x_test_padded, imdb_y_test)
  print('loss: {} accuracy: {}'.format(*model_2_scores))

782/782 [==============================] - 64s 81ms/step - loss: 0.3004 - binary_accuracy: 0.8770
loss: 0.3003838360309601 accuracy: 0.8770400285720825


## Model 3
- Removing an LSTM layer from the model has a negligible impact on accuracy.

In [49]:
model_3 = Sequential()
model_3.add(Embedding(input_dim=len(imdb.get_word_index()) + imdb_index_offset,
                              output_dim=8,
                              input_length=cutoff))
# return_sequences tells the LSTM to output the full sequence, for use by the next LSTM layer. The final
# LSTM layer should return only the output sequence, for use in the Dense output layer
# model_3.add(LSTM(units=32, return_sequences=True))
model_3.add(LSTM(units=32))
model_3.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

with device('/device:GPU:0'):
  model_3.fit(imdb_x_train_padded, imdb_y_train, epochs=2, batch_size=64)

Epoch 1/2
391/391 [==============================] - 68s 170ms/step - loss: 0.5765 - binary_accuracy: 0.6646
Epoch 2/2
391/391 [==============================] - 67s 170ms/step - loss: 0.2058 - binary_accuracy: 0.9264


In [50]:
with device('/device:GPU:0'):
  model_3_scores = model_3.evaluate(imdb_x_test_padded, imdb_y_test)
  print('loss: {} accuracy: {}'.format(*model_3_scores))

782/782 [==============================] - 28s 36ms/step - loss: 0.2962 - binary_accuracy: 0.8776
loss: 0.29620546102523804 accuracy: 0.8776000142097473


## Model 4

- Reduced the number of LSTM units to 8, with a negligible impact on training accuracy.

In [53]:
model_4 = Sequential()
model_4.add(Embedding(input_dim=len(imdb.get_word_index()) + imdb_index_offset,
                              output_dim=8,
                              input_length=cutoff))
# return_sequences tells the LSTM to output the full sequence, for use by the next LSTM layer. The final
# LSTM layer should return only the output sequence, for use in the Dense output layer
# model_3.add(LSTM(units=32, return_sequences=True))
model_4.add(LSTM(units=4))
model_4.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
model_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

with device('/device:GPU:0'):
  model_4.fit(imdb_x_train_padded, imdb_y_train, epochs=2, batch_size=64)

Epoch 1/2
391/391 [==============================] - 55s 136ms/step - loss: 0.6415 - binary_accuracy: 0.6438
Epoch 2/2
391/391 [==============================] - 53s 136ms/step - loss: 0.3876 - binary_accuracy: 0.8668


In [52]:
with device('/device:GPU:0'):
  model_4_scores = model_4.evaluate(imdb_x_test_padded, imdb_y_test)
  print('loss: {} accuracy: {}'.format(*model_4_scores))

782/782 [==============================] - 22s 28ms/step - loss: 0.3458 - binary_accuracy: 0.8583
loss: 0.3458023965358734 accuracy: 0.8583199977874756


# Exercise Option #2 - Advanced Difficulty

Set up your own RNN model for the Reuters Classification Problem

Take the model from exercise 1 (imdb_lstm_model) and modify it to classify the [Reuters data](https://keras.io/datasets/#reuters-newswire-topics-classification).

Think about what you are trying to predict in this case, and how you will have to change your model to deal with this.

In [ ]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
(reuters_x_train, reuters_y_train), (reuters_x_test, reuters_y_test) = reuters.load_data()

2113536/2110848 [==============================] - 0s 0us/step
